In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [2]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [3]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [4]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [5]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [6]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [7]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d9452203-Reviews-Shilla_Stay_Gwanghwamun-Seoul.html"
url2 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d8118189-Reviews-or240-Shilla_Stay_Mapo-Seoul.html"
url3 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d7054545-Reviews-Shilla_Stay_Yeoksam-Seoul.html"
url4 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d9454537-Reviews-Shilla_Stay_Guro-Seoul.html"

data_sample = hotel_review_crawling(url4)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
26 번째 페이지
27 번째 페이지
28 번째 페이지
29 번째 페이지
30 번째 페이지
31 번째 페이지
32 번째 페이지
33 번째 페이지
34 번째 페이지
35 번째 페이지
36 번째 페이지
37 번째 페이지
38 번째 페이지
39 번째 페이지
40 번째 페이지
41 번째 페이지
42 번째 페이지
43 번째 페이지
44 번째 페이지
45 번째 페이지
46 번째 페이지
47 번째 페이지
48 번째 페이지
49 번째 페이지
50 번째 페이지
51 번째 페이지
52 번째 페이지
53 번째 페이지
54 번째 페이지
55 번째 페이지
56 번째 페이지
57 번째 페이지
58 번째 페이지
59 번째 페이지
60 번째 페이지
61 번째 페이지
62 번째 페이지
63 번째 페이지
64 번째 페이지
65 번째 페이지
66 번째 페이지
67 번째 페이지
68 번째 페이지
69 번째 페이지
70 번째 페이지
71 번째 페이지
72 번째 페이지
73 번째 페이지


nickname score                                             review  \
0          Jae    50  작은 테이블도 있어서 뭐 먹기에도 좋구요\n\n어메니티도 구비잘되어 있어요\n\n적...   
1         ming    40  생각보다 조금 작지만 침대가 푹신하고 역이랑 편의점이 가까워서 좋아요😊😊 에어컨 빵...   
2    BBBJJJHHH    50  7월 중순에 4박정도 이용했습니다.\n신라스테이를 고른 이유는 평균 1박에 8만원 ...   
3          Leg    50  볼일이 있어 기대 없이 호텔을 예약을 하고 왔는데 정말 잘 쉬다 가네요 , 시국이 ...   
4       eunseo    50  2박 하면서 불편한 점 없이 너무 잘 쉬다 갑니다! 직원분들도 너무 친절하셨고 룸 ...   
..         ...   ...                                                ...   
360     jun651    40  적당한 가격과 객실상태도 만족하엿습니다. 어메니티도 좋았구요^^ㅎㅎㅎ\n주변에 식당...   
361        무호2    50  새배드림 패키지로 왔다. 호텔 가격이 저렴하고 기념품으로 곰인형을 주었다. 점심 저...   
362        SHL    10  호텔 시설은 모던하고 깔끔한데, 관리가 전혀 안됨. 하우스키퍼 퇴근한 6시 이후로는...   
363       1234    10  전에도 이용하고 불친절해서 불쾌했던 곳인데 약속장소와 가까워서 다시 이용해보니 여전...   
364        arl    40  신라스테이 계열은 인테리어도 비슷ㅎㅎ 그냥 모던한 느낌이다 조식은 꼭 먹기를 메뉴는...   

      stay_date  
0    2021-07-01  
1    2021-07-01  
2    2021-07-01  
3    2021-07-01  
4    2021-06-01  
..          ...  
360  2019-01-01  
361  2019-02-01  
362  2019-02-01  
363  2019-02-01  
364  2018-03-01  

[365 rows x 4 columns]

In [7]:
data_sample = pd.read_csv('./data//hotel_review_shillastay_yeoksam_final.csv')
data_sample = data_sample[['nickname', 'score', 'review', 'stay_date']]

In [8]:
data_sample.stay_date = data_sample.apply(lambda X :datetime.datetime.strptime(X.stay_date,'%Y-%m-%d'), axis=1)

In [9]:
data_sample.stay_date = data_sample.apply(lambda X : X.stay_date.date(), axis=1)

In [10]:
data_sample

nickname  score                                             review  \
0          Park     50  시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...   
1         Mochi     50  직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다 무엇보다도 위치부분에서...   
2           이가현     50  방도 깨끗하고 직원들도 너무 친절했습니다 무엇보다 호텔의 위치가 너무 좋아서 서울에...   
3       jakekim     50  직원들도 너무 친정하고 깕끔해 가성비 그자체 강남에 이만한 호텔은 없는것 같습니다 ...   
4          비갠오후     40  며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다 인테리어도 상당히 깔끔하고 북적거리...   
..          ...    ...                                                ...   
91       Nexwho     50  휴가로 하루밤을 묵었습니다  호텔은 역삼역과 선릉역 사이에 위치하고 있어 교통이 편...   
92  Lee seokhee     50  무료 룸 업그레이드도 받고 객실도 깔끔하고 전체적으로 만족스러웠습니다 배달 음식 시...   
93       cookki     50  업무상 며칠 묵었습니다 대로변에 있어서 찾기 쉽더군요  깔끔하고 식사도 좋네요 특히...   
94   Kimwonseob     50  정말 오랜만에 이정도 가격에 가성비에 최고인가같아요짱이에여 다음에 또방문할거같아용 ...   
95   Baekdoodan     40  오랫만에 투숙하게된 신라스테이 역삼 다른데는 모르겠고 역삼점은 제일 깔끔하고 깨끗한...   

     stay_date  
0   2021-06-01  
1   2021-06-01  
2   2021-05-01  
3   2021-05-01  
4   2021-03-01  
..         ...  
91  2019-08-01  
92  2019-08-01  
93  2019-08-01  
94  2019-08-01  
95  2019-08-01  

[96 rows x 4 columns]

In [11]:
# 중간에 크롤링이 끊겼을 경우
url_next = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d7054545-Reviews-or100-Shilla_Stay_Yeoksam-Seoul.html"
data_sample2 = hotel_review_crawling(url_next)

data_sample_concat = pd.concat([data_sample, data_sample2], axis=0, ignore_index=True)
data_sample_concat

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
26 번째 페이지
27 번째 페이지
28 번째 페이지
29 번째 페이지


nickname score  \
0                Park    50   
1               Mochi    50   
2                 이가현    50   
3             jakekim    50   
4                비갠오후    40   
..                ...   ...   
236      Dbenterbiz89    50   
237         sunkkot87    50   
238             Sol Y    20   
239  BackPacker769583    50   
240          JAEHHYUK    50   

                                                review   stay_date  
0    시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...  2021-06-01  
1    직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다 무엇보다도 위치부분에서...  2021-06-01  
2    방도 깨끗하고 직원들도 너무 친절했습니다 무엇보다 호텔의 위치가 너무 좋아서 서울에...  2021-05-01  
3    직원들도 너무 친정하고 깕끔해 가성비 그자체 강남에 이만한 호텔은 없는것 같습니다 ...  2021-05-01  
4    며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다 인테리어도 상당히 깔끔하고 북적거리...  2021-03-01  
..                                                 ...         ...  
236  사업 미팅 차 강남에서 투숙해야만 했던 상황에 신라스테이 역삼에서 투숙했습니다. 호...  2018-04-01  
237  제가 웬만하면 후기를 남기지 않지만, 저와같은 비즈니스맨에게 추천하기위해 몇자 남깁...  2018-04-01  
238  늦은체크인시 직원분은 프론트옆에 있는 바 치우느라\n손님을 봤는데도 치우는데 열중\...  2018-04-01  
239  가족을 만날일이나 업무로 서울에 갈때마다 늘 망설임 없이 선택하는 신라스테이 역삼!...  2018-04-01  
240  지난 주말에 신라 스테이 역삼 디럭스룸을 이용했습니다.\n\n룸 규모는 커진 않지만...  2018-04-01  

[241 rows x 4 columns]

In [12]:
data_sample_concat

nickname score  \
0                Park    50   
1               Mochi    50   
2                 이가현    50   
3             jakekim    50   
4                비갠오후    40   
..                ...   ...   
236      Dbenterbiz89    50   
237         sunkkot87    50   
238             Sol Y    20   
239  BackPacker769583    50   
240          JAEHHYUK    50   

                                                review   stay_date  
0    시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...  2021-06-01  
1    직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다 무엇보다도 위치부분에서...  2021-06-01  
2    방도 깨끗하고 직원들도 너무 친절했습니다 무엇보다 호텔의 위치가 너무 좋아서 서울에...  2021-05-01  
3    직원들도 너무 친정하고 깕끔해 가성비 그자체 강남에 이만한 호텔은 없는것 같습니다 ...  2021-05-01  
4    며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다 인테리어도 상당히 깔끔하고 북적거리...  2021-03-01  
..                                                 ...         ...  
236  사업 미팅 차 강남에서 투숙해야만 했던 상황에 신라스테이 역삼에서 투숙했습니다. 호...  2018-04-01  
237  제가 웬만하면 후기를 남기지 않지만, 저와같은 비즈니스맨에게 추천하기위해 몇자 남깁...  2018-04-01  
238  늦은체크인시 직원분은 프론트옆에 있는 바 치우느라\n손님을 봤는데도 치우는데 열중\...  2018-04-01  
239  가족을 만날일이나 업무로 서울에 갈때마다 늘 망설임 없이 선택하는 신라스테이 역삼!...  2018-04-01  
240  지난 주말에 신라 스테이 역삼 디럭스룸을 이용했습니다.\n\n룸 규모는 커진 않지만...  2018-04-01  

[241 rows x 4 columns]

In [13]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample_concat.to_csv('./data/hotel_review_shillastay_yeoksam_origin.csv', encoding="utf-8-sig")

In [14]:
data = preprocessing_data(data_sample_concat)

In [15]:
data

nickname score                                             review  \
0             Park    50  시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...   
1            Mochi    50  직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다 무엇보다도 위치부분에서...   
2              이가현    50  방도 깨끗하고 직원들도 너무 친절했습니다 무엇보다 호텔의 위치가 너무 좋아서 서울에...   
3          jakekim    50  직원들도 너무 친정하고 깕끔해 가성비 그자체 강남에 이만한 호텔은 없는것 같습니다 ...   
4             비갠오후    40  며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다 인테리어도 상당히 깔끔하고 북적거리...   
..             ...   ...                                                ...   
231         ansclf    50  먼저 체크인 시, 친절하게 도와주신 Edward Moon 직원분께 감사드려요ㅎㅎ 덕...   
232       sssyyy13    50  신라스테이 역삼 디럭스더블룸에서 투숙했습니다. 처음 로비 왔을때 부터 모던한 분위기...   
233        ksy2023    50  출장으로 인해 투숙했습니다. 객실도 깔끔 하고 몇일 있는 동안 만족스런 투숙이었네요...   
234  kimsarang2018    50  삼성동에서 월드컵 응원 하고 잘곳을 급하게 찾다가 1박 했습니다. 르메르디앙 예약하...   
235       Jongsuki    20  회사 단체 미팅으로 연회장에서 식사를 하러갔는데 신라계열이라고 들어서 크게 기대하고...   

      stay_date  
0    2021-06-01  
1    2021-06-01  
2    2021-05-01  
3    2021-05-01  
4    2021-03-01  
..          ...  
231  2018-06-01  
232  2018-06-01  
233  2018-06-01  
234  2018-06-01  
235  2018-06-01  

[236 rows x 4 columns]

In [16]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-16-7183346d9643>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [17]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-17-b71e843350f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [18]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_yeoksam_final.csv', encoding="utf-8-sig")

<ipython-input-18-a2b534a793de>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [19]:
test_data

nickname score                                             review  \
0             Park    50  시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...   
1            Mochi    50  직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다 무엇보다도 위치부분에서...   
2              이가현    50  방도 깨끗하고 직원들도 너무 친절했습니다 무엇보다 호텔의 위치가 너무 좋아서 서울에...   
3          jakekim    50  직원들도 너무 친정하고 깕끔해 가성비 그자체 강남에 이만한 호텔은 없는것 같습니다 ...   
4             비갠오후    40  며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다 인테리어도 상당히 깔끔하고 북적거리...   
..             ...   ...                                                ...   
231         ansclf    50  먼저 체크인 시 친절하게 도와주신 Edward Moon 직원분께 감사드려요 덕분에 ...   
232       sssyyy13    50  신라스테이 역삼 디럭스더블룸에서 투숙했습니다 처음 로비 왔을때 부터 모던한 분위기에...   
233        ksy2023    50  출장으로 인해 투숙했습니다 객실도 깔끔 하고 몇일 있는 동안 만족스런 투숙이었네요 ...   
234  kimsarang2018    50  삼성동에서 월드컵 응원 하고 잘곳을 급하게 찾다가 1박 했습니다 르메르디앙 예약하려...   
235       Jongsuki    20  회사 단체 미팅으로 연회장에서 식사를 하러갔는데 신라계열이라고 들어서 크게 기대하고...   

      stay_date  
0    2021-06-01  
1    2021-06-01  
2    2021-05-01  
3    2021-05-01  
4    2021-03-01  
..          ...  
231  2018-06-01  
232  2018-06-01  
233  2018-06-01  
234  2018-06-01  
235  2018-06-01  

[236 rows x 4 columns]